In [1]:
from __future__ import unicode_literals, print_function

import pandas as pd
from collections import Counter
import spacy,re,json
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy  

from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

import re
from pprint import pprint
import os
import warnings
warnings.filterwarnings('ignore')

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

# ============================================================
## 基于 Spacy 的 NER 模型

In [2]:
PATH ='NER_dataset/file_ner.json'

annotated_data = []
lines=[]
with open(PATH, 'r',encoding = 'utf-8 sig') as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['text']
    entities = []
    for label in data['labels']:
        entities.append(label)
    annotated_data.append((text, {"entities": data['labels']}))

In [3]:
training_data=  annotated_data[0:200]
test_data = annotated_data[200:220]

In [4]:
print(len(training_data), len(test_data))

200 20


In [5]:
training_data[0]

('"Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company\'s growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (

In [6]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in training_data:
        doc = nlp(text)
#         print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#         print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in training_data:
            doc = nlp2(text)
#             print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#             print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# trained this model with 100 iterations in google colab using GPU runtime with loss: Losses {'ner': 465.24571515528373} 
main(output_dir='temp_model/') 

Created blank 'en' model
Losses {'ner': 15591.360430225335}
Losses {'ner': 2787.020977496403}
Losses {'ner': 2774.859541359775}
Losses {'ner': 2628.9528426013776}
Losses {'ner': 2550.4716138278554}
Losses {'ner': 2291.53565723599}
Losses {'ner': 2279.1441345600615}
Losses {'ner': 2149.2046633712625}
Losses {'ner': 2102.456777007096}
Losses {'ner': 1970.2895380651523}
Losses {'ner': 1915.8428007980824}
Losses {'ner': 1732.1216285954558}
Losses {'ner': 1738.5568885237362}
Losses {'ner': 1738.4657628273492}
Losses {'ner': 1666.4754492796062}
Losses {'ner': 1717.1825983124968}
Losses {'ner': 1631.3995744115718}
Losses {'ner': 1579.1631971586514}
Losses {'ner': 1558.5411924776176}
Losses {'ner': 1585.0137448119694}
Losses {'ner': 1565.202226530998}
Losses {'ner': 1361.8915551126504}
Losses {'ner': 1361.8010196534613}
Losses {'ner': 1435.3527402853583}
Losses {'ner': 1346.0797067944748}
Losses {'ner': 1267.2114909524423}
Losses {'ner': 1409.9519635288466}
Losses {'ner': 1143.2423026348106}
L

# ==========================================================
## 导入 NER 模型，识别 JD 中的所有技能类关键词

In [12]:
ner_model = spacy.load('temp_model/')

In [8]:
skill_set = {}
link_set = {}

In [9]:
ind = 0

for root,dirs,files in os.walk(r"../JD_files"):
    for file in files:
        ind += 1
        
        try:
            #获取文件路径
            my_file = open( os.path.join(root,file),encoding = 'utf-8' )
            my_string = my_file.readlines()

            for string in my_string[1:]:
                doc = ner_model( string )

                temp_list = []
                for chunk in doc.ents:
                    temp_list.append( chunk.text )
                    
                temp_set = set( temp_list )
                
                for tem in list(temp_set):
                    if tem not in skill_set.keys():
                        print( tem )
                        
                    if skill_set.get( tem, 0 ) == 0:
                        skill_set[tem] = 1
                    else:
                        skill_set[tem] += 1
                        
                for t1 in list(temp_set):
                    for t2 in list(temp_set):
                        if t1.text != t2.text:
                            if link_set.get( t1+'|'+t2, 0 ) == 0:
                                link_set[t1+'|'+t2] = 1
                            else:
                                link_set[t1+'|'+t2] += 1
                            

            my_file.close()
        
            if ind % 100 == 0:
                print( '------------------------------------------------------------------------',ind )
                
#             if ind > 2000:
#                 break
                
        except:
            pass

Potential

Data Engineer
MapReduce

Python
operations analysis
R
consumer analytics
Apache Spark
ElasticSearch
Snowflake
PostgreSQL
BigQuery
Assist sales
Google Analytics
Background

Deep Learning experience
Project Management
Hadoop
and/or scripting
Programmer certification
structures

Python/PySpark

Axiologic Solutions

Django
Machine Learning
Machine learning
SAS
machine learning
Predictive Analytics
Position Overview

Excel
including Word
SAP
deep learning
social network analysis
Map/Reduce
C++
DigitalOcean
S3
JavaScript
Spark
Redshift
MySQL
Hive
Java
Crimson Hexagon
Boosting
C
TensorFlow
Kafka
Deep learning
consulting
Data
AWZ
Databricks
Detroit
Michigan
Deep Learning
Strong scripting language
Sas
Industries 

analytics experience
Data Processing
ETL
Data Analytics
Data Scientist
required; MA
artificial intelligence
required
problem-solving
you

SQL
Analytics
Data Mining

Data Modeling
Programming experience
Python experience
MS Office
SAP Hana
EC2
Team

SPSS
Data Analysis
artifi

Terraform
PowerShell
Azure Data Lake Analytics
DevOps
Octopus Deploy
Azure Analysis
Microsoft BI
Statistical modeling
multivariate testing
forecasting/predictive analytics
SCAN

Strong SQL

and/or SQL

Advanced degrees
SK
pathway
Data Manipulation
JMP
automation

Microsoft Excel
Microsoft SQL
PowerPivot
Advanced Standard
Github Experience
Microsoft office
network analysis
Project Leader
Optional deferred compensation
SQL;

DevOps Familiar
MSSQL Experience
IC GovCLoud
Hypothesis testing
Text analysis
Neo4j

and/or Python -Strong quantitative analysis
machine learningapproaches
non-linear
WORLD
data modeling
e.g. scripting languages
Texas

DATA SCIENTIST
RESEARCH DEPARTMENT
Lead

Design

Spark experience
Strong math
software engineering
numerical linear algebra
; experience
Deep Learning networks
XML Schema
audit
Unit testing
Integration testing
Contribution

Power Point
GRAPH
systems
STAT
scripting language
Exceptional relationship building
DB2
MS SQL Server
Requirement

project managem

MATLAB;

Access

Lambda
Envoy

Data Workbench
clickstream analysis
non-SQL
MySQL

UT 84020 

Draper
Data

Team Overview

Client Analytics
Inventory
including SAS
advanced SQL experience
NIELSEN

​
ScalaStrong
MS Azure
PIG
Modelling-190019850

Audit
credit bureaus
SASUnix
IFRS9
database query languages
Office 365
Microsoft Advertising
.NET
We are a successful
troubleshooting
Targeting
GGlpot
Map Reduce
Sagemaker
Python andor R
using scripting
Life Sciences
Jave
Theano
BioInformatics
Graph)Creating
PeopleSoft
Business Analysis
javascript
css
etc;

anomaly detection
Mathematical Engineering
Pension

Data gap analysis
NLP

Public Affairs
Stata
Intelligence

PTO

--------------------------------------------

Tensorflow

Google)
NLTK

predictive trend analysis Experience
and/or SQL Experience
Kinesis
Warehouse
GCP / NumPy / SQL
Data analysis experience
design patterns
unsupervised feature learning
reinforcement learning
Deep reinforcement learning
Swift

machine
Unix experience
Cassandra
htm

-----------------------

Miner
graph analytics
Alpine Data Labs
As a 

59282

Neo4j
for predictive analytics
Data Sciences
SAS Studio
GRAPHICS
SAS BI/Dashboard
portfolio risk management
PYTHON and/or R
etc.-
Microsoft T-SQL
Delighters

R and/or
statistical algorithms
SPECIAL ACCOUNTABILITY

Unlimited Vacation

Provides support
Predictive model building
MACHINE LEARNING 

Regression
SQL AWS
Trees
text mining
SSH
Local Recruitment

OPE001RORequisition # 

Programming
Thorough understanding
Six-Sigma
Powerpoint
Data Office
Consumer FY
+3
------------------------------------------------------------------------ 7400
SAP experience
Java Experience
ABILITIES REQUIRED Experience
MS SQL Server; CPA
Iron Python
get things
USC

Six Sigma Master
data visualization 

Data Engineer

Data Analyts
1

viewing of
VB
customer care, services
credit research reports
Python JavaScript
Kong 0.10.3
------------------------------------------------------------------------ 7600
SageMaker
Cosmo DB
Cognitive Scien

In [14]:
skill_set = dict( Counter(skill_set).most_common(300) )
skill_set

{'Python': 1570,
 'R': 1077,
 'machine learning': 820,
 'SQL': 768,
 'SAS': 532,
 'Hadoop': 463,
 'Machine Learning': 420,
 'Java': 368,
 'Spark': 362,
 'C++': 303,
 'Industries \n': 201,
 'Deep Learning': 200,
 'C': 187,
 'SPSS': 143,
 'AWS': 141,
 'problem-solving': 134,
 'problem solving': 127,
 'Requirements\n': 116,
 'e.g.': 109,
 'Analytics': 101,
 'NoSQL': 98,
 'TensorFlow': 85,
 'MATLAB': 85,
 'MapReduce\n': 83,
 'Machine learning': 81,
 'JavaScript': 79,
 'Perl': 79,
 'deep learning': 78,
 'analytics': 76,
 'Matlab': 73,
 'ETL': 72,
 'S3': 70,
 'Artificial Intelligence': 70,
 'C#': 65,
 'Linux': 64,
 'predictive modeling': 64,
 'Azure': 62,
 'artificial intelligence': 61,
 'predictive analytics': 60,
 'Data Analytics': 59,
 'Excel': 58,
 'Oracle': 57,
 'Tableau': 57,
 'NLP': 56,
 'Data Lake': 55,
 'research': 55,
 'Data Analysis': 53,
 'SQL\n': 53,
 'PHP': 53,
 'Data Scientist': 52,
 'EC2': 51,
 'MySQL': 47,
 'Tensorflow': 47,
 'Power BI': 47,
 'Microsoft Office': 46,
 'Postgr

# ============================================================
## 基于句法分析抽取层级关系

### 根据每个技能类关键词，向上找最小分支树，判断依存关系

In [15]:
nlp = spacy.load("en_core_web_sm")

In [16]:
def is_father( word, root ):
    if word == root:
        return True
    
    while word != word.head:
        word = word.head
        if word == root:
            return True
    
    return False

In [23]:
ind = 0

total_link_list = []
link_set = []

for root,dirs,files in os.walk(r"../JD_files"):
    for file in files:
        ind += 1
            
#         if ind % 2 == 1:
#             continue
        
        my_file = open( os.path.join(root,file),encoding = 'utf-8' )

        my_string = my_file.readlines()
        for string_list in my_string[1:]:
            string_list = string_list.replace('e.g.','like')
            string_list = string_list.replace('.)',')')
#             for string in [sentence for sentence in re.split(r'[？?！!。；;：:\n\r·]', string_list) if sentence]:
            for string in re.split(r'[？?！!。.；;：:\n\r·•*]', string_list):
#                 try:
                doc = nlp( string )

                my_chunks_list = []
                roor_chunks_dict = {}

                for chunk in doc.noun_chunks:
                    if chunk.text in skill_set.keys():
                        my_chunks_list.append( chunk.root )
                        roor_chunks_dict[ str(chunk.root)] = str(chunk)

                if len( my_chunks_list ) > 1:
#                     print( 'skill word:', my_chunks_list )
                    Total_List = []

                    for word in my_chunks_list:
                        temp_list = []
                        temp_list.append(word)

                        pre = word
                        while word != word.head:
                            pre = word
                            relation = word.dep_
                            word = word.head

                            if 'N' in word.tag_ and (relation != 'conj' and relation != 'appos'):
                                if word in my_chunks_list:
                                    temp_list.append( word )

                                    if len( [str( roor_chunks_dict[str(w)] ) for w in temp_list if 'xperience' not in str(w)] ) == 2:
#                                             print(string)
#                                             print('result1:') 
                                            if ' --> '.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) not in link_set: 
                                                print( ' --> '.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )
                                                total_link_list.append( [str( roor_chunks_dict[str(w)] ) for w in temp_list] )
                                                link_set.append( ' --> '.join([str( roor_chunks_dict[str(w)] ) for w in temp_list]) )

                                else:
                                    tree_list = []
                                    for t in my_chunks_list:
                                        if t.dep_ == 'compound' and is_father( t, word ) == True and is_father( t, pre ) == False:
                                            tree_list.append( t )

                                    if len(tree_list) == 1:
                                        key1 = roor_chunks_dict[ str(temp_list[0]) ]
                                        key2 = roor_chunks_dict[ str(tree_list[0]) ]
                                        
                                        if 'xperience' not in key1 and 'xperience' not in key2:
#                                             print(string)
#                                             print('result2:')  
                                            if ' --> '.join( [ key1, key2 ] ) not in link_set: 
                                                print( ' --> '.join( [ key1, key2 ] ) )
                                                total_link_list.append( [ key1, key2 ] )
                                                link_set.append( ' --> '.join( [ key1, key2 ] ) )
                                break
                
        my_file.close()

        if ind % 500 == 0:
            print( '==================================================',ind )
            
#         if ind > 2000:
#             break

R --> scripting
Python --> scripting
predictive modeling --> advanced analytics
machine learning --> advanced analytics
data visualization --> advanced analytics
MatLab --> Weka
Flask --> analysis
Word --> MS Office
Excel --> MS Office
SQL Server --> MS Office
PowerPoint --> MS Office
Machine Learning --> Data Science
Data Warehouse --> AWS
SAS --> Statistical analysis
R --> Statistical analysis
Python --> Statistical analysis
national origin --> sex
================================================== 500
Deep Learning --> Deep Learning
NoSQL --> databases
C# --> scripting
Java --> scripting
Perl --> scripting
Tableau --> data visualization
data visualization --> data analytics
visualization --> R
NoSQL --> Hive
SQL --> relational databases
deep learning --> deep learning
research --> database
================================================== 1000
Scala --> Java
SQL --> SAS
Scala --> Python
machine learning --> C++
================================================== 1500
SAS --> scripti

# DataFrame

In [26]:
link_df = pd.DataFrame( total_link_list )
link_df = link_df.rename(columns = {0:'node1',1:'node2'})

# 关系绘制

In [27]:
from unittest.mock import patch
from nose.tools import assert_equal, assert_in
from pyecharts import options as opts
from pyecharts.charts.basic_charts.graph import Graph

from pyecharts import options as opts
from pyecharts.charts import Graph

nodes = []
links = []

nodes_set = []

for ind, row in link_df.iterrows():
    
    if row['node1'] != row['node2']:
        
        if row['node1'] not in nodes_set:
            nodes_set.append( row['node1'] )
            nodes.append( {'name': row['node1'] , "symbolSize": 15 + skill_set[row['node1']] / 20 } )
            
        if row['node2'] not in nodes_set:
            nodes_set.append( row['node2'] )
            nodes.append( {'name': row['node2'] , "symbolSize": 15 + skill_set[row['node2']] / 20 } )
        
        links.append({"source": row['node2'], "target": row['node1']})

Graph().add(
    "", 
    nodes, 
    links, 
    repulsion=8000,
#     linestyle_opts=opts.LineStyleOpts(width=2, curve=0.1, opacity=0.9),
    linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
    label_opts=opts.LabelOpts( ),
    categories=None,
    is_focusnode=True,
#     layout="none",
    is_roam=True
    
).set_global_opts(title_opts=opts.TitleOpts(title="")).render_notebook()

/home/lothar/anaconda3/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
